### Single Shot Detector (SSD)- Dynamic environment

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# COCO class names (80 classes + background)
COCO_CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 
    'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 
    'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 
    'N/A', 'backpack', 'umbrella', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 
    'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 
    'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 
    'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
    'teddy bear', 'hair drier', 'toothbrush'
]

# Load the SSD MobileNet model from local directory
def load_ssd_model():
    """Load the SSD MobileNet model from a local path."""
    model_path = "C:\\Users\\chari\\Downloads\\ssd-mobilenet-v2-tensorflow2-ssd-mobilenet-v2-v1"
    model = tf.saved_model.load(model_path)
    return model

# Perform object detection using SSD
def detect_objects(model, image_tensor):
    """Perform object detection using SSD model."""
    detections = model(image_tensor)
    return {
        "detection_boxes": detections["detection_boxes"],
        "detection_scores": detections["detection_scores"],
        "detection_classes": detections["detection_classes"],
    }

# Draw bounding boxes and labels
def draw_detections(frame, detections, threshold=0.5):
    """Draw bounding boxes and labels on the frame."""
    img_height, img_width, _ = frame.shape

    detection_boxes = detections["detection_boxes"].numpy()[0]
    detection_scores = detections["detection_scores"].numpy()[0]
    detection_classes = detections["detection_classes"].numpy()[0].astype(np.int32)

    for box, score, cls in zip(detection_boxes, detection_scores, detection_classes):
        if score > threshold:
            ymin, xmin, ymax, xmax = box
            xmin, xmax = int(xmin * img_width), int(xmax * img_width)
            ymin, ymax = int(ymin * img_height), int(ymax * img_height)

            # Validate class index
            if 0 <= cls < len(COCO_CLASSES):
                class_name = COCO_CLASSES[cls]
            else:
                class_name = "Unknown"

            # Draw bounding box and label
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            label = f"{class_name}: {score:.2f}"
            cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame


# Live object detection using SSD model
def dynamic_object_detection():
    """Capture video and apply SSD object detection dynamically."""
    print("Loading SSD model...")
    model = load_ssd_model()
    print("Model loaded successfully.")

    cap = cv2.VideoCapture(0)  # Use webcam; replace '0' with video file path for custom video
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    print("Press 'q' to quit.")
    plt.ion()  # Turn on interactive mode for Matplotlib
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Preprocess the frame
        frame_resized = cv2.resize(frame, (320, 320))  # SSD input size
        input_tensor = tf.convert_to_tensor(frame_resized, dtype=tf.uint8)  # Ensure uint8 type
        input_tensor = tf.expand_dims(input_tensor, axis=0)  # Add batch dimension

        # Perform detection
        detections = detect_objects(model, input_tensor)

        # Draw detections on the frame
        frame_with_detections = draw_detections(frame, detections)

        # Use Matplotlib to display the frame
        plt.imshow(cv2.cvtColor(frame_with_detections, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.pause(0.001)  # Pause to update the plot
        plt.clf()  # Clear the figure for the next frame

    cap.release()
    plt.close()

# Run the dynamic object detection
if __name__ == "__main__":
    dynamic_object_detection()
